https://www.maricopacountyattorney.org/CivicAlerts.aspx?AID=400

Crime reports?


In [ ]:
var fs = require('fs');
var path = require('path');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = path.join(PROFILE_PATH, 'Collections/crimes');

function scrapeAlert(ID) {
    if(fs.existsSync(path.join(project, 'maricopa-alert-' + ID + '.json'))) {
        return;
    }
    return client
        .url('https://www.maricopacountyattorney.org/CivicAlerts.aspx?AID=' + ID)
        .getAllXPath({
            time: '//*[@class = "single"]//*[@class = "date"]//text()',
            title: '//*[contains(@class, "single")]//h3//text()',
            content: '//*[@class = "single"]//*[@class = "content"]//text()'
        })
        .then(r => {
            fs.writeFileSync(path.join(project, 'maricopa-alert-' + ID + '.json'), JSON.stringify(r, null, 4));
            return r;
        })
        .catch(e => console.log(e))
}
module.exports = scrapeAlert;

if(typeof $$ !== 'undefined') {
    $$.async();
    var IDs = Array.from(Array(500).keys());
    multiCrawl(IDs, 'crime reports')
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


multi crawl?


In [ ]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

var TIMEOUT = 1000;
var CONNECTIONS = 3;

// recursively dequeue tasks
function deQueue(inputQueue, searchCallback, ctx) {
    const results = [];
    console.log(ctx.client.requestHandler.sessionID);
    const callback = typeof searchCallback === 'function'
        ? searchCallback
        : importer.import(searchCallback, Object.assign({useCache: false}, ctx));
    if(inputQueue.length > 0) {
        const item = inputQueue.shift();
        return new Promise(resolve => setTimeout(() => resolve(), 100))
            .then(() => callback(...[item, ctx]))
            .catch(e => {
                console.log(e + '');
                if((e + '').indexOf('Already') > -1 || (e + '').indexOf('session') > -1) {
                    inputQueue.push(item);
                    throw new Error('Abandoning session :(', e);
                }
            })
            .then(r => results.push(r))
            .then(() => deQueue(inputQueue, searchCallback, ctx))
            .then(r => results.concat(r))
            .catch(e => console.log(e))
    } else {
        return results;
    }
}

// create a number of individual selenium sessions and dequeue the tasks with the callback search
function multiCrawl(inputList, searchCallback) {
    var indexes = Array.from(Array(Math.min(inputList.length, CONNECTIONS)).keys());
    var connections = [];
    var promises = indexes.map((s, i) => resolve => {
        const client = runSeleniumCell(false, false);
        return client
            // skip this if error
            //.then(() => connections[i].onlyOneWindow())
            //.then(() => connections[i].resizeWindow())
            .then(ctx => {
                connections.push(ctx);
                resolve(ctx)
            })
            .catch(e => {
                console.log(e);
                resolve(null);
            })
    });
    var queue = [].concat(inputList);
    var count = 0;
    return importer.runAllPromises(promises)
        .then(() => {
            console.log(connections.map(c => c.client.requestHandler.sessionID));
        })
        .then(() => {
            return connections[0].client
                .scanning(true)
                .then(() => connections[0].getAllSessionUrls())
                .scanning(false)
        })
        .then(() => console.log('done loading sessions'))
        .then(() => Promise.all(connections.map(ctx => deQueue(queue, searchCallback, ctx))))
        .then(r => [].concat([], ...r))
}
module.exports = multiCrawl;



search results as json?

In [ ]:
function searchResultsToJson(url) {
    console.log(url + ' - ' + client.requestHandler.sessionID);
    return client
        .url(url)
        .pause(2000)
        .getAllXPath({
            query: '//input[contains(@aria-label, "Search")]/@value'
            +
            '|//input[contains(@aria-label, "search")]/@value'
            +
            // yahoo
            '|//label[contains(., "Search")]/following::*//input[@type="text"]/@value' 
            +
            '|//input[contains(@class, "Search")]/@value'
            +
            // wolfram
            '|//input[contains(@name, "query")]/@value'
            +
            // duckduckgo
            '|//input[contains(@id, "search")]/@value'
            +
            // yandex
            '|//input[contains(@aria-label, "Request")]/@value',
            results: [
                '//h3|//h2|div[contains(@class, "title")]'
                +
                // ask
                '|//*[contains(@class, "item-title")]',
                {
                    name: './/text()',
                    summary: './/following-sibling::div//text()'
                }
            ]
        })
        .then(r => {
            return {
                url: url,
                query: typeof r.query === 'string'
                    ? r.query
                    : r.query[0],
                results: r.results.map(s => ({
                    name: typeof s.name === 'string'
                        ? s.name : s.name.join('\n'),
                    summary: typeof s.summary === 'string'
                        ? s.summary : s.summary.join('\n')
                }))
            };
        })
        .catch(e => {
            console.log(e)
            return {
                url: url,
                query: null,
                results: []
            }
        })
}
module.exports = searchResultsToJson;


meta search all?


In [ ]:
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var multiCrawl = importer.import('multi crawl');

// http://www.exploratorium.edu/files/ronh/research/index.html
/*

 Exploratorium	
 Search
 Google	
 Search
 alltheweb	
 Search
 Teoma	
 Search
 AltaVista	
 Search
 Wisenut	
 Search
 HotBot	
 Search
 lii.org	
 Search
 Northern Light	
 Search
 Lycos	
 Search
 Scirus	
 
 Meta:
  Ask Jeeves	
 Search
 MetaCrawler	
 Search
 Dogpile	
 Search
 SavvySearch	
 
 
 */
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = path.join(PROFILE_PATH, 'Collections/searches');

function searchAll(query = 'search engine') {
    var engines = [
        'https://www.google.com/search?q=' + query,
        'https://www.bing.com/search?q=' + query,
        'https://search.yahoo.com/search?p=' + query,
        'https://www.ask.com/web?q=' + query,
        'https://search.aol.com/aol/search?q=' + query,
        'http://www.baidu.com/s?wd=' + query,
        'https://www.wolframalpha.com/input/?i=' + query,
        'https://duckduckgo.com/?q=' + query,
        'https://www.yandex.com/search/?text=' + query,
        'https://archive.org/search.php?query=' + query,
    ];
    
    // TODO: save results
    return multiCrawl(engines, 'search results json')
        .then(r => {
            const time = new Date();
            fs.writeFileSync(path.join(project, query.replace(/[^a-z0-9]/ig, '_')
                                       + '-' + time.getFullYear()
                                       + '-' + (time.getMonth() + 1)
                                       + '-' + time.getDate()
                                       + '.json'), JSON.stringify(r, null, 4));
            return r;
        })
}
module.exports = searchAll;

if(typeof $$ !== 'undefined') {
    $$.async();
    searchAll()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


schedule search all?



In [ ]:
var google = require('googleapis');
var importer = require('../Core');
var {
    getOauthClient,
    ISODateString,
    createNewEvent,
} = importer.import([
    'convert date iso',
    'create new calendar event',
    'import google calendar api',
]);

var options = {
    calendarId: 'aws'
}

function scheduleSearch(search) {
    const parameters = {
        query: search || 'search engines'
    }
    const newDate = new Date();
    return (typeof options.auth === 'undefined'
           ? getOauthClient(options)
           : Promise.resolve([]))
        .then(() => createNewEvent('meta search all', JSON.stringify(parameters, null, 4), options))
}
module.exports = scheduleSearch;


crawl domain?



In [ ]:
var fs = require('fs');
var path = require('path');
var _ = require('lodash');
var importer = require('../Core');
var multiCrawl = importer.import('multi crawl');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = path.join(PROFILE_PATH, 'Collections/crawls');

function startCrawl(entries) {
    return multiCrawl(entries, (url, {client, getAllXPath}) => {
        var result = {};
        console.log(url);
        return client
            .url(url)
            .pause(2000)
            .then(() => getAllXPath({
                links: '//a/@href|//*[@src]/@src',
                styles: '//link/@href|style/@text()'
            }))
            .then(r => Object.assign(result, r))
            .getHTML('body')
            .then(html => Object.assign(result, {url: url, content: html}))
    });
}

function fixLink(url, l) {
    return l.trim().match(/^\/\//ig)
       ? ('https:' + l.trim())
       : (l.trim().match(/^https?:\/\//ig)
         ? l.trim()
         : url.replace(/\/$/ig, '') + '/' + l.trim().replace(/^\//ig, ''));
}

function crawlAll(url) {
    url = url || 'https://www.act.com';
    var entries = [
        url
    ];
    
    // TODO: save results
    return startCrawl(entries)
        .then(searches => {
            // TODO: download cache images / styles in gist?
            entries = _.uniq(entries.concat(...searches.map(s => {
                // TODO: replace links in document
                return s.styles.map(l => fixLink(url, l))
                    .concat(s.links.map(l => fixLink(url, l)));
            })));
            return startCrawl(entries);
        })
        .then(searches => {
            const time = new Date();
            fs.writeFileSync(path.join(project, url.replace(/[^a-z0-9]/ig, '_')
                                       + '-' + time.getFullYear()
                                       + '-' + (time.getMonth() + 1)
                                       + '-' + time.getDate()
                                       + '.json'), JSON.stringify(searches, null, 4));
            return searches;
        });
}
module.exports = crawlAll;

if(typeof $$ !== 'undefined') {
    $$.async();
    crawlAll()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}



schedule crawl domain?



In [ ]:
var google = require('googleapis');
var importer = require('../Core');
var {
    getOauthClient,
    ISODateString,
    createNewEvent,
} = importer.import([
    'convert date iso',
    'create new calendar event',
    'import google calendar api',
]);

var options = {
    calendarId: 'aws'
}

function scheduleSearch(search) {
    const parameters = {
        query: search
    }
    const newDate = new Date();
    return (typeof options.auth === 'undefined'
           ? getOauthClient(options)
           : Promise.resolve([]))
        .then(() => createNewEvent('crawl domain', JSON.stringify(parameters, null, 4), options))
}
module.exports = scheduleSearch;


tell joke?



In [ ]:
var util = require('bluebird');
var request = util.promisify(require('request'));

function regexToArray(ex, str, i = 0) {
    var co = []; var m;
    while ((m = ex.exec(str)) && co.push(m[i]));
    return co;
}

var jokes;
function getJoke() {
    // TODO: collect jokes instead
    return (typeof jokes === 'undefined'
        ? request('http://www.ducksters.com/jokes/silly.php')
        .then(res => regexToArray(/^.*?Q:.*$|^.*?A:.*$/igm, res.body))
        .then(r => {
            r = r.reduce((arr, j, i) => {
                if(i % 2 === 1) {
                    arr.push([
                        r[i-1].replace(/<.*?\s*\/?>/ig, '').trim().replace(/^\s*|\s*$/igm, ''),
                        j.replace(/<.*?\s*\/?>/ig, '').trim().replace(/^\s*|\s*$/igm, '')
                    ]);
                }
                return arr;
            }, []);
            console.log(r);
            jokes = r;
            return r;
        })
        : Promise.resolve(jokes))
        .then(arr => {
            const i = Math.round(Math.random() * arr.length);
            return arr[i];
        })
}
module.exports = getJoke;

